In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cvxpy
import multiprocessing as mp
from tqdm import tqdm

In [2]:
import io_utils
import kernels
import algos

In [3]:
raw_train, raw_val, labels_train, labels_val = [], [], [], []

for i in range(3):
    r_tr, r_val, _, _, lbl_tr, lbl_val = io_utils.train_val_single_dataset(i)
    raw_train.append(r_tr)
    raw_val.append(r_val)
    labels_train.append(lbl_tr)
    labels_val.append(lbl_val)

In [ ]:
pool = mp.Pool(6)
# Compute kernels
kernel_train, kernel_vals = [], []

kern_ss = kernels.SubstringKernel(7, 0.1)

for n in range(3):
    n_train, n_val = raw_train[n].shape[0], raw_val[n].shape[0]
    
    t = time.time()
    args = [(raw_train[n][i], raw_train[n][j]) for i in range(n_train) for j in range(i, n_train) ]
    result = pool.map(kern_ss.computeKernel, args)
    e = time.time()
    print(e-t)
    
    count = 0
    K = np.zeros((n_train, n_train))

    for i in range(n_train):
        for j in range(i, n_train):
            K[i,j] = result[count]
            count +=1

    for i in range(n_train):
        for j in range(0, i):
            K[i,j] = K[j,i]
            args = [(raw_train[n][i], raw_val[n][j]) for i in range(n_train) for j in range(n_val) ]
    
    result = pool.map(kern_ss.computeKernel, args)
    intermediate = np.zeros((n_train, n_val))
    count = 0
    for i in tqdm(range(n_train)):
        for j in range(n_val):
            intermediate[i,j] = result[count]
            count +=1
    kernel_train.append(K.copy())
    kernel_vals.append(intermediate.copy())



In [ ]:
# Test
kernel_tests = []

for i in range(3):
    cur_test = io_utils.load_raw_test(i)
    
    args = [(raw_train[i][j], cur_test[k]) for j in range(raw_train[i].shape[0]) for k in range(cur_test.shape[0]) ]
    result = pool.map(computeKernel, args)

    kernel_test = np.zeros((raw_train[i].shape[0], cur_test.shape[0]))

    count = 0
    for k in range(raw_train[i].shape[0]):
        for l in range(cur_test.shape[0]):
            kernel_test[k,l] = result[count]
            count +=1
            
    kernel_tests.append(kernel_test.copy())

In [4]:
# Load from files
kernel_train, kernel_vals, kernel_tests = io_utils.load_kernels("kernels/substring_7_01")

In [5]:
## Prediction 
lambd = 7e-2
all_preds = []

krr = algos.KernelRidgeRegression()

for i in range(3):
    krr.fit(kernel_train[i], labels_train[i], lambd)
    error = krr.evaluatePerformance(kernel_vals[i], labels_val[i])
  
    # Test
    pred_test = krr.predict(kernel_tests[i])
    all_preds.append(pred_test.copy())


Score : 0.458
Score : 0.485
Score : 0.493


In [6]:
lambda_vals = np.logspace(-2, 3, TRIALS)

krr = algos.KernelRidgeRegression()
for i in range(3):
    errors = krr.searchBestParameters(kernel_train[i], train_labels[i], kernel_vals[i], val_labels[i], lambda_vals)

NameError: name 'importlib' is not defined